In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
#from src.scripts.process_tweets import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
from collections import defaultdict
import seaborn as sns
%matplotlib inline

In [2]:
# Loop through cities
tweet_dir= Path("../data/processed/park_user_tweets_0530/")
cities = list(tweet_dir.glob("*.json"))

## load all stop wordds
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)

In [104]:
## test time control 
park_tweet_text, control_tweet_text, park_list = get_time_control_text(park_user_tweets)
park_stops  = get_park_stopwords(park_list)
stop_list = park_stops+stop_dict['all_cities']+ stop_dict[city.stem]
bootstrap_sentiment(park_tweet_text, control_tweet_text, stop_list,sample=.8,runs=5)

[0.2649276604737896,
 0.2703131775603964,
 0.2762774607979246,
 0.27388624292304176,
 0.2730704345396955]

In [105]:
### test user control
park_tweet_text, control_tweet_text, park_list = get_user_control_text(park_user_tweets)
park_stops  = get_park_stopwords(park_list)
stop_list = park_stops+stop_dict['all_cities']+ stop_dict[city.stem]
bootstrap_sentiment(park_tweet_text, control_tweet_text, stop_list,sample=.8,runs=5)

[0.17795150667741133,
 0.18529823330290363,
 0.18207451565484334,
 0.1809849095152778,
 0.1921141852609436]

In [3]:
results_dict = {}

In [4]:
tweet_dir= Path("../data/processed/park_user_tweets_0530/")
for city in tweet_dir.glob("*.json"):
    if city.name not in results_dict.keys():
        with open(city, 'r') as f:
            park_user_tweets = json.load(f)
        print(city.name, len(park_user_tweets))
        park_tweet_text, control_tweet_text, park_list = get_time_control_text(park_user_tweets)
        park_stops  = get_park_stopwords(park_list)
        stop_list = park_stops+stop_dict['all_cities']+ stop_dict[city.stem]
        stop_lower = [x.lower() for x in dict(regex.get_ngrams(' '.join(stop_list), path='../src/func/ngrams.bin')).keys()]
        park_sentis, control_sentis = bootstrap_sentiment(park_tweet_text, control_tweet_text, stop_lower,sample=.8,runs=10, bumps=False)
        results_dict[city.name]={"park_sentiment":park_sentis,"control_sentiment":control_sentis}

CO_Denver_0820000.json 3902
AZ_Phoenix_0455000.json 7566
FL_Jacksonville_1235000.json 3218
TX_Austin_4805000.json 14689
TX_Fort_Worth_4827000.json 4278
TX_El_Paso_4824000.json 1397
CA_San_Diego_0666000.json 22269
TN_Memphis_4748000.json 3112
DC_Washington_1150000.json 41062
WA_Seattle_5363000.json 7739
IL_Chicago_1714000.json 36919
TX_Houston_4835000.json 13464
MA_Boston_2507000.json 23479
MI_Detroit_2622000.json 3819
NC_Charlotte_3712000.json 3868
NY_New_York_3651000.json 113702
IN_Indianapolis_1836003.json 5660
TX_San_Antonio_4865000.json 12763
TX_Dallas_4819000.json 12211
OH_Columbus_3918000.json 4340
PA_Philadelphia_4260000.json 26287
CA_Los_Angeles_0644000.json 36271
MD_Baltimore_2404000.json 5135
CA_San_Jose_0668000.json 4517
CA_San_Francisco_0667000.json 36175


In [6]:
with open("./results/bootstrap_time_0530.json", 'w') as fp:
    json.dump(results_dict,fp)

In [7]:
results_u_dict = {}

In [8]:
tweet_dir= Path("../data/processed/park_user_tweets_0530")
for city in tweet_dir.glob("*.json"):
    if city.name not in results_u_dict.keys():
        with open(city, 'r') as f:
            park_user_tweets = json.load(f)
        print(city.name, len(park_user_tweets))
        park_tweet_text, control_tweet_text, park_list = get_user_control_text(park_user_tweets)
        park_stops  = get_park_stopwords(park_list)
        stop_list = park_stops+stop_dict['all_cities']+ stop_dict[city.stem]
        stop_lower = [x.lower() for x in dict(regex.get_ngrams(' '.join(stop_list), path='../src/func/ngrams.bin')).keys()]
        park_sentis, control_sentis = bootstrap_sentiment(park_tweet_text, control_tweet_text, stop_lower,sample=.8,runs=10, bumps=False)
        results_u_dict[city.name]={"park_sentiment":park_sentis,"control_sentiment":control_sentis}

CO_Denver_0820000.json 3902
AZ_Phoenix_0455000.json 7566
FL_Jacksonville_1235000.json 3218
TX_Austin_4805000.json 14689
TX_Fort_Worth_4827000.json 4278
TX_El_Paso_4824000.json 1397
CA_San_Diego_0666000.json 22269
TN_Memphis_4748000.json 3112
DC_Washington_1150000.json 41062
WA_Seattle_5363000.json 7739
IL_Chicago_1714000.json 36919
TX_Houston_4835000.json 13464
MA_Boston_2507000.json 23479
MI_Detroit_2622000.json 3819
NC_Charlotte_3712000.json 3868
NY_New_York_3651000.json 113702
IN_Indianapolis_1836003.json 5660
TX_San_Antonio_4865000.json 12763
TX_Dallas_4819000.json 12211
OH_Columbus_3918000.json 4340
PA_Philadelphia_4260000.json 26287
CA_Los_Angeles_0644000.json 36271
MD_Baltimore_2404000.json 5135
CA_San_Jose_0668000.json 4517
CA_San_Francisco_0667000.json 36175


In [9]:
with open("./results/bootstrap_user_0530.json", 'w') as fp:
    json.dump(results_u_dict,fp)

## sensitivity tests and stop word check ##

In [10]:
# Now we bootstrap a vector of 100 seniments, by choosing 80% of the tweets in each group
for n in np.arange(.1,1,.1):
    bumps = bootstrap_sentiment(park_tweet_text, control_tweet_text, stop_list,sample=n,runs=5)
    print("{:.0f}% sampled: Mean Park: {:.4f} with range {:.4f}".format(n*100, np.mean(bumps), max(bumps)-min(bumps)))
#print(min(park_sentis), np.mean(park_sentis), max(park_sentis))
#print(min(control_sentis), np.mean(control_sentis), max(control_sentis))

10% sampled: Mean Park: 0.0173 with range 0.0625
20% sampled: Mean Park: 0.0326 with range 0.0731
30% sampled: Mean Park: 0.0257 with range 0.0619
40% sampled: Mean Park: 0.0207 with range 0.0437
50% sampled: Mean Park: 0.0255 with range 0.0370
60% sampled: Mean Park: 0.0207 with range 0.0249
70% sampled: Mean Park: 0.0234 with range 0.0280
80% sampled: Mean Park: 0.0227 with range 0.0167
90% sampled: Mean Park: 0.0226 with range 0.0040


In [137]:
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang='english',returnVector=True)
stop_lower = [x.lower() for x in dict(regex.get_ngrams(' '.join(stop_list), path='../src/func/ngrams.bin')).keys()]
for word in stop_lower:
    if word in labMTwordList:
        i = labMTwordList.index(word)
        val = labMTvector[i]
        if val < 4 or val > 6:
            pass
            #print(word, labMTvector[i])